# Pré processamento: 
Funções do arquivo pre_processing.py

In [44]:
import numpy as np
import pandas as pd
import nltk
import pre_processing as pp
nltk.download('punkt')
nltk.download('stopwords')
category = 'test'

df, df_chi2, Y = pp.bow(category)
X = df.values
X_chi = df_chi2.values

[nltk_data] Downloading package punkt to /home/sy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
    abl  absolut  accomplish  acquir  action  actual  acupunctur  adam  advic  \
0   0.0      0.0         0.0     0.0     0.0     1.0         0.0   0.0    0.0   
1   0.0      0.0         1.0     0.0     0.0     0.0         1.0   0.0    0.0   
2   0.0      0.0         0.0     0.0     0.0     0.0         0.0   0.0    0.0   
3   0.0      0.0         0.0     0.0     0.0     0.0         0.0   0.0    0.0   
4   0.0      0.0         0.0     0.0     0.0     0.0         0.0   0.0    0.0   
5   0.0      0.0         0.0     0.0     0.0     1.0         0.0   0.0    0.0   
6   0.0      0.0         0.0     0.0     0.0     0.0         0.0   0.0    0.0   
7   0.0      0.0         0.0     0.0     0.0     0.0         0.0   0.0    0.0   
8   0.0      0.0         0.0     0.

Holdout estratificado para separar dados para treino e teste:

In [8]:
train_index, test_index = pp.stratified_holdOut(Y, 0.3)

In [9]:
#Treinamento
Xtrain = X[train_index]
Xtrain_chi = X_chi[train_index]
Ytrain = Y[train_index]

#Validação
Xval =  X[test_index]
Xval_chi = X_chi[test_index]
Yval = Y[test_index]

# K-vizinhos:

In [19]:
%load_ext autoreload
%autoreload 2

import k_neighbors as knn

for j in range(3, 8, 2):
    
    K = j
    
    print("K = " + str(K))
    
    classes = []

    for i in range(Xval.shape[0]):
        y = knn.knn(Xval[i], Xtrain, Ytrain, K)
        classes.append(y)

    acuracia = np.sum(classes==Yval)/len(Yval)
    print("Acuracia sem chi-squared é " + str(acuracia))    

    classes = []
    
    for i in range(Xval.shape[0]):
        y = knn.knn(Xval_chi[i], Xtrain_chi, Ytrain, K)
        classes.append(y)
        
    acuracia = np.sum(classes==Yval)/len(Yval)
    print("Acuracia com chi-squared é " + str(acuracia))    
    
    print("-------------------------------------")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
K = 3
Acuracia sem chi-squared é 0.5
Acuracia com chi-squared é 0.6666666666666666
-------------------------------------
K = 5
Acuracia sem chi-squared é 0.5
Acuracia com chi-squared é 0.6666666666666666
-------------------------------------
K = 7
Acuracia sem chi-squared é 0.5
Acuracia com chi-squared é 0.7222222222222222
-------------------------------------


# Regressão Logística

In [14]:
%load_ext autoreload
%autoreload 2

import regressao_logistica as rl

import scipy.optimize 

# Configura se a função custo utiliza a regularização
reg = 0
# Configura o parametro de regularizacao lambda igual a 1
lambda_reg = 1
# Algumas configuracoes do gradiente descente
iteracoes = 50
theta = np.zeros(Xval.shape[1]) # Inicializa parâmetros que serao ajustados

# minimiza a funcao de custo
result = scipy.optimize.minimize(fun=rl.funcaoCusto, x0=theta, args=(Xtrain, Ytrain,reg,lambda_reg),  
                method='BFGS', jac=True, options={'maxiter': iteracoes, 'disp':True})
# coleta os thetas retornados pela função de minimização
theta = result.x

# realiza a predição dos dados
p = rl.predicao(theta, Xval)

acuracia = np.mean(p == Yval) * 100
print("Acuracia sem chi-squared é " + str(acuracia)) 

theta = np.zeros(Xval_chi.shape[1]) # Inicializa parâmetros que serao ajustados

# minimiza a funcao de custo
result = scipy.optimize.minimize(fun=rl.funcaoCusto, x0=theta, args=(Xtrain_chi, Ytrain,reg,lambda_reg),  
                method='BFGS', jac=True, options={'maxiter': iteracoes, 'disp':True})
# coleta os thetas retornados pela função de minimização
theta = result.x

# realiza a predição dos dados
p = rl.predicao(theta, Xval_chi)

acuracia = np.mean(p == Yval) * 100
print("Acuracia com chi-squared é " + str(acuracia)) 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 19
         Function evaluations: 20
         Gradient evaluations: 20
Acuracia sem chi-squared é 55.5555555556
Optimization terminated successfully.
         Current function value: 0.346580
         Iterations: 17
         Function evaluations: 18
         Gradient evaluations: 18
Acuracia com chi-squared é 66.6666666667


# Naive Bayes

In [10]:
%load_ext autoreload
%autoreload 2

import naive_bayes as nb

#Sem chi
probPos, probNeg, pPos, pNeg = nb.calcularProbabilidades(Xtrain, Ytrain)
classes = []

for i in range(Xval.shape[0]):
    y = nb.classificacao(Xval[i], probPos, probNeg, pPos, pNeg)
    classes.append(y)

acuracia = np.sum(classes==Yval)/len(Yval)
print("Acuracia sem chi-squared é " + str(acuracia))    

#Com chi
probPos, probNeg, pPos, pNeg = nb.calcularProbabilidades(Xtrain_chi, Ytrain)
classes = []
for i in range(Xval.shape[0]):
    y = nb.classificacao(Xval_chi[i], probPos, probNeg, pPos, pNeg)
    classes.append(y)

acuracia = np.sum(classes==Yval)/len(Yval)
print("Acuracia com chi-squared é " + str(acuracia))    


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Acuracia sem chi-squared é 0.5
Acuracia com chi-squared é 0.7777777777777778


# Redes Neurais Artificiais

# SVM

# Maximum Entropy Classifier

In [48]:
from nltk.classify import MaxentClassifier

def word_feats(words):
    return dict([(word, True) for word in words])

negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

negcutoff = int(len(negfeats)*3/4)
poscutoff = int(len(posfeats)*3/4)

trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

training_set = trainfeats
test_set = testfeats

#for i in range(len(Ytrain)) :
#    training_set.append((Xtrain_chi[i,:].tolist(), Ytrain[i]))
#for i in range(len(Yval)) :    
#    test_set.append((Xval_chi[i,:].tolist(), Yval[i]))

#def list_to_dict(words_list):
#  return dict([(word, True) for word in words_list])

training_set_formatted = [(list_to_dict(element[0]), element[1]) for element in training_set] 
numIterations = 100
 
algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifier = nltk.MaxentClassifier.train(training_set_formatted, algorithm, max_iter=numIterations)
#classifier.show_most_informative_features(10)

test_set_formatted = [(list_to_dict(element[0]), element[1]) for element in test_set] 
classes = []
for review in test_set_formatted:
    label = review[1]
    text = review[0]
    classes.append(classifier.classify(text))
    
acuracia = np.sum(classes==Yval)/len(Yval)
print(acuracia)

[nltk_data] Downloading package movie_reviews to /home/sy/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
<bound method CategorizedCorpusReader.fileids of <CategorizedPlaintextCorpusReader in '/home/sy/nltk_data/corpora/movie_reviews'>>


# Random Forest

In [53]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(Xtrain_chi, Ytrain)
classes = clf.predict(Xval_chi)
acuracia = np.sum(classes==Yval)/len(Yval)
print("Acuracia com chi-squared é " + str(acuracia))  

Acuracia com chi-squared é 0.7777777777777778


# INVENTAR +++